In [296]:
import pandas as pd
import geopandas as gp
import numpy as np
import censusgeocode as cg
import time
import os

In [244]:
census_tracts = gp.read_file('/Users/ameliabaum/Desktop/Amelia/Berkeley 2018-2019/EnRes C190/evictionmodel_190final/cb_2017_06_tract_500k/cb_2017_06_tract_500k.shp')
sf_tracts = census_tracts[census_tracts['COUNTYFP'] == '075']
sf_tracts.head() #196 census tracts in SF


,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
430,06,075,010200,1400000US06075010200,06075010200,102,CT,515959,295388,"POLYGON ((-122.426671 37.809639, -122.424876 3..."
431,06,075,011200,1400000US06075011200,06075011200,112,CT,177415,0,"POLYGON ((-122.416292 37.793892, -122.415218 3..."
432,06,075,012401,1400000US06075012401,06075012401,124.01,CT,91972,0,"POLYGON ((-122.417707 37.784236, -122.416072 3..."
433,06,075,013300,1400000US06075013300,06075013300,133,CT,581310,0,"POLYGON ((-122.459476 37.789711, -122.45274 37..."
434,06,075,015900,1400000US06075015900,06075015900,159,CT,296386,0,"POLYGON ((-122.434562 37.784152, -122.431291 3..."


In [245]:
sf_evictions = pd.read_csv("/Users/ameliabaum/Desktop/Amelia/Berkeley 2018-2019/EnRes C190/evictionmodel_190final/data/Eviction_Notices.csv")
sf_evictions["Year"] = sf_evictions["File Date"].astype(str).apply(lambda row: row[6:])
sf_evictions["Year"] = sf_evictions["Year"].astype(int)
sf_evictions_11 = sf_evictions[sf_evictions["Year"] >= 2011].dropna()
sf_evictions_11.head()

/Users/ameliabaum/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Eviction ID,Address,City,State,Eviction Notice Source Zipcode,File Date,Non Payment,Breach,Nuisance,Illegal Use,...,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Constraints Date,Supervisor District,Neighborhoods - Analysis Boundaries,Location,Year
6,M172623,100 Block Of Charles Street,San Francisco,CA,94131,10/19/2017,False,False,False,False,...,False,False,False,False,False,12/17/2022,8.0,Glen Park,"(37.738336951880164, -122.4256608010048)",2017
10,M172628,1300 Block Of Chestnut Street,San Francisco,CA,94123,10/23/2017,False,False,False,False,...,False,False,False,False,False,12/17/2022,2.0,Marina,"(37.80223066530217, -122.42556524759928)",2017
14,M172503,1200 Block Of 41st Avenue,San Francisco,CA,94122,10/02/2017,False,False,False,False,...,False,False,False,False,False,11/29/2022,4.0,Sunset/Parkside,"(37.76346716977734, -122.50069529991866)",2017
22,M172511,0 Block Of 5th Avenue,San Francisco,CA,94118,10/10/2017,False,False,False,False,...,False,False,False,False,False,12/05/2022,2.0,Inner Richmond,"(37.787705866671075, -122.46361972068956)",2017
25,M172709,100 Block Of Prospect Avenue,San Francisco,CA,94110,10/26/2017,False,False,False,False,...,False,False,False,False,False,12/24/2022,9.0,Bernal Heights,"(37.74296049255459, -122.41894065545237)",2017


In [246]:
#split the lat and lon into different columns
sf_evictions_11["Latitude"] = sf_evictions_11["Location"].astype(str).apply(lambda row: row.split(",")[0][1:])
sf_evictions_11["Longitude"] = sf_evictions_11["Location"].astype(str).apply(lambda row: row.split(",")[1][:-1])

sf_evictions_11

,Eviction ID,Address,City,State,Eviction Notice Source Zipcode,File Date,Non Payment,Breach,Nuisance,Illegal Use,...,Lead Remediation,Development,Good Samaritan Ends,Constraints Date,Supervisor District,Neighborhoods - Analysis Boundaries,Location,Year,Latitude,Longitude
6,M172623,100 Block Of Charles Street,San Francisco,CA,94131,10/19/2017,False,False,False,False,...,False,False,False,12/17/2022,8.0,Glen Park,"(37.738336951880164, -122.4256608010048)",2017,37.738336951880164,-122.4256608010048
10,M172628,1300 Block Of Chestnut Street,San Francisco,CA,94123,10/23/2017,False,False,False,False,...,False,False,False,12/17/2022,2.0,Marina,"(37.80223066530217, -122.42556524759928)",2017,37.80223066530217,-122.42556524759928
14,M172503,1200 Block Of 41st Avenue,San Francisco,CA,94122,10/02/2017,False,False,False,False,...,False,False,False,11/29/2022,4.0,Sunset/Parkside,"(37.76346716977734, -122.50069529991866)",2017,37.76346716977734,-122.50069529991866
22,M172511,0 Block Of 5th Avenue,San Francisco,CA,94118,10/10/2017,False,False,False,False,...,False,False,False,12/05/2022,2.0,Inner Richmond,"(37.787705866671075, -122.46361972068956)",2017,37.787705866671075,-122.46361972068956
25,M172709,100 Block Of Prospect Avenue,San Francisco,CA,94110,10/26/2017,False,False,False,False,...,False,False,False,12/24/2022,9.0,Bernal Heights,"(37.74296049255459, -122.41894065545237)",2017,37.74296049255459,-122.41894065545237
30,M172508,400 Block Of Staples Avenue,San Francisco,CA,94112,10/04/2017,False,False,False,False,...,False,False,False,01/01/2023,7.0,West of Twin Peaks,"(37.72900535140412, -122.45178538729365)",2017,37.72900535140412,-122.45178538729365
32,M172625,100 Block Of Cordova Street,San Francisco,CA,94112,10/23/2017,False,False,False,False,...,False,False,False,12/12/2022,11.0,Excelsior,"(37.71193832413155, -122.43563244891334)",2017,37.71193832413155,-122.43563244891334
39,M172507,5500 Block Of California Street,San Francisco,CA,94121,10/04/2017,False,False,False,False,...,False,False,False,11/28/2022,1.0,Outer Richmond,"(37.784191971543926, -122.47696912767651)",2017,37.784191971543926,-122.47696912767651
43,M172501,1500 Block Of Fulton Street,San Francisco,CA,94117,10/02/2017,False,False,False,False,...,False,False,False,11/29/2022,5.0,Lone Mountain/USF,"(37.776276302674454, -122.44229944730586)",2017,37.776276302674454,-122.44229944730586
44,M172627,2500 Block Of Quintara Street,San Francisco,CA,94116,10/23/2017,False,False,False,False,...,False,False,False,12/18/2022,4.0,Sunset/Parkside,"(37.747857144692475, -122.49268820953169)",2017,37.747857144692475,-122.49268820953169


In [247]:
def getTract(lon, lat):
#pings the FCC API to geocode lat lon coordinates to other geograpies. Returns a 2 item array of [geoid, tract name]
    try:
        result = cg.coordinates(lon, lat)
        tract = result['Census Tracts'][0]['NAME'] #alt use 'TRACT' for partial fips
        return tract
    except(ValueError, TypeError):
        return 'Nan'


In [248]:
def getGeoid(lon, lat):
#pings the FCC API to geocode lat lon coordinates to other geograpies. Returns a 2 item array of [geoid, tract name]
    try:
        result = cg.coordinates(lon, lat)
        geoid = result['Census Tracts'][0]['GEOID']
        return geoid
    except(ValueError, TypeError):
        return 'Nan'
        

In [249]:
start = time.time()
sf_evictions_11["Geoid"] = sf_evictions_11.apply(lambda row: getGeoid(row["Longitude"], row["Latitude"]), axis=1)
end = time.time()
elapsed = end-start #this takes like ~30 mins



In [235]:
# sf_evictions_11["Tract"] = sf_evictions_11.apply(lambda row: getTract(row["Longitude"], row["Latitude"]), axis=1)

Series([], dtype: object)

In [252]:
sf_evictions_11.to_csv("data/evictions_geocided_11.csv")

In [267]:
evictions_geocoded11_tractandyear = sf_evictions_11.groupby(["Geoid", 
        "Year"]).count().drop(["Neighborhoods - Analysis Boundaries", "Location", "Latitude", "Longitude", 
        "Eviction Notice Source Zipcode", "Address", "City", "State", "Constraints Date", "File Date", "Supervisor District"], axis=1) #166 census tracts represented

In [ ]:
evictions_geocoded11_tractandyear.to_csv("data/evictions_geocoded11_tractandyear.csv")

In [295]:
evictions_geocoded11_tractandyear = pd.read_csv("data/evictions_geocoded11_tractandyear.csv")
evictions_geocoded11_tractandyear["Tract"] = evictions_geocoded11_tractandyear["Geoid"].astype(str).apply(lambda row: row[5:])
evictions_geocoded11_tractandyear["tract_year"] = evictions_geocoded11_tractandyear["Tract"] + evictions_geocoded11_tractandyear["Year"].astype(str)
evictions_geocoded11_tractandyear.to_csv("data/evictions_geocoded11_tractandyear.csv")                                                                                               
                                                                                                          
                                                                                                                                                                                                                                                                                                               
                                                                                                          
                                                                                                                                                                                                                                                   
                                                                                                          

In [325]:
data_path = '/Users/ameliabaum/Desktop/Amelia/Berkeley 2018-2019/EnRes C190/evictionmodel_190final/data/'
#table B02001 race
race_feature = {}
for race_file in os.listdir(data_path+'B02001_race'):
    year = race_file.split("_")[1]
    race_feature[year] = pd.read_csv(data_path+'B02001_race/'+race_file)
print(race_feature.keys())


#table B25071 rent burden
burden_feature = {}
for burden_file in os.listdir(data_path+'B25071_burden'):
    year = burden_file.split("_")[1]
    if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
        burden_feature[year] = pd.read_csv(data_path+'B25071_burden/'+burden_file)
print(burden_feature.keys())

#table S1101 household size
hsize_feature = {}
for size_file in os.listdir(data_path+'S1101_size'):
    year = size_file.split("_")[1]
    if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
        hsize_feature[year] = pd.read_csv(data_path+'S1101_size/'+size_file)
print(hsize_feature.keys())

#table S1501 % with bachelors degree
edu_feature = {}
for edu_file in os.listdir(data_path+'S1501_edu'):
    year = edu_file.split("_")[1]
    if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
        edu_feature[year] = pd.read_csv(data_path+'S1501_edu/'+edu_file)
print(edu_feature.keys())

#table B02001 race
race_feature = {}
for race_file in os.listdir(data_path+'B02001_race'):
    year = race_file.split("_")[1]
    if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
        race_feature[year] = pd.read_csv(data_path+'B02001_race/'+race_file)
print(race_feature.keys())



#table S1101 household size
size_feature = {}
for size_file in os.listdir(data_path+'S1101_size'):
    year = size_file.split("_")[1]
    if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
        size_feature[year] = pd.read_csv(data_path+'S1101_size/'+size_file)
print(size_feature.keys())

#table B25035 median year built
built_feature = {}
for built_file in os.listdir(data_path+'B25035_yearbuilt'):
    year = built_file.split("_")[1]
    if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
        built_feature[year] = pd.read_csv(data_path+'B25035_yearbuilt/'+built_file)
print(built_feature.keys())


#table B25039 median renter moved in
moved_feature = {}
for moved_file in os.listdir(data_path+'B25039_moved'):
    year = moved_file.split("_")[1]
    if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
        moved_feature[year] = pd.read_csv(data_path+'B25039_moved/'+moved_file)
print(moved_feature.keys())


#table B25001 housing units in tract
units_feature = {}
for units_file in os.listdir(data_path+'B25001_units'):
    year = units_file.split("_")[1]
    if year in ['09', '10', '11', '12', '13', '14', '15', '16']:
        units_feature[year] = pd.read_csv(data_path+'B25001_units/'+units_file)
print(units_feature.keys())




dict_keys(['15', '12', '10', '11', '16', '13', '09', '14'])
dict_keys(['09', '15', '10', '11', '14', '12', '13', '16'])
dict_keys(['13', '09', '12', '16', '14', '10', '11', '15'])
dict_keys(['16', '12', '11', '09', '15', '13', '14', '10'])
dict_keys(['15', '12', '10', '11', '16', '13', '09', '14'])
dict_keys(['13', '09', '12', '16', '14', '10', '11', '15'])
dict_keys(['10', '16', '11', '14', '13', '09', '12', '15'])
dict_keys(['09', '13', '14', '15', '12', '10', '11', '16'])
dict_keys(['15', '12', '13', '09', '14', '11', '16', '10'])
